In [1]:
"""Installations----"""
!pip install --q unstructured langchain
!pip install langchain
# !pip install pikepdf
# !pip install --q "unstructured[all-docs]"

In [2]:
# from langchain_community.document_loaders import UnstructuredPDFLoader
# # from langchain_community.document_loaders import OnlinePDFLoader
# import os
from flask import Flask, jsonify, request
import os
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate

In [3]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [4]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED       
nomic-embed-text:latest	0a109f422b47	274 MB	20 seconds ago	
llama3:latest          	365c0bd3c000	4.7 GB	13 days ago   	


In [14]:
!pwd

/Users/admin/Documents/workspace/venv/Rag_Automation/RAG_prototype


In [20]:
import os
import pdfplumber

_path = "/Users/admin/Documents/pdf_file_blobs"
local_path = _path + "/" + os.listdir(_path)[0]

loader = PDFPlumberLoader(local_path)
docs = loader.load_and_split()
print(docs[0].page_content)

Digitally enabled business models for a
circular plastic economy in Africa
Kolade, O., Oyinlola, M., Ogunde, O., Ilo, C. & Ajala, O.
Published PDF deposited in Coventry University’s Repository
Original citation:
Kolade, O, Oyinlola, M, Ogunde, O, Ilo, C & Ajala, O 2024, 'Digitally enabled business
models for a circular plastic economy in Africa', Environmental Technology &
Innovation, vol. 35, 103657.
https://dx.doi.org/10.1016/j.eti.2024.103657
DOI 10.1016/j.eti.2024.103657
ISSN 2352-1864
Publisher: Elsevier
This is an open access article under the CC BY license
(http://creativecommons.org/licenses/by/4.0/)


In [31]:
import os
from langchain_community.document_loaders import PDFPlumberLoader
import pandas as pd


def PdfFiles(_path):
    files = os.listdir(_path)
    df = pd.DataFrame(files, columns=["file_name"])


    df["desktop_path"] = _path + "/" + df["file_name"]
    file_paths = df["desktop_path"].values
    for j in file_paths:
        loader = PDFPlumberLoader(j)
        docs = loader.load_and_split()
        return docs[0].page_content


_path = "/Users/admin/Documents/pdf_file_blobs"
print(PdfFiles(_path))


Does R&D investment drive employment
growth? Empirical evidence at industry
level from Japan
Shah, I. H., Kollydas, K., Lee, P. Y., Malki, I. & Chu, C.
Published PDF deposited in Coventry University’s Repository
Original citation:
Shah, IH, Kollydas, K, Lee, PY, Malki, I & Chu, C 2022, 'Does R&D investment drive
employment growth? Empirical evidence at industry level from Japan', International
Journal of Finance & Economics, vol. (In-press), pp. (In-press).
https://dx.doi.org/10.1002/ijfe.2677
DOI 10.1002/ijfe.2677
ISSN 1076-9307
ESSN 1099-1158
Publisher: Wiley
© 2022 The Authors. International Journal of Finance & Economics published by
John Wiley & Sons Ltd.
This is an open access article under the terms of the Creative Commons Attribution
License, which permits use, distribution and reproduction in any medium, provided
the original work is properly cited.


In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
print(chunks)

[Document(page_content="Digitally enabled business models for a\ncircular plastic economy in Africa\nKolade, O., Oyinlola, M., Ogunde, O., Ilo, C. & Ajala, O.\nPublished PDF deposited in Coventry University’s Repository\nOriginal citation:\nKolade, O, Oyinlola, M, Ogunde, O, Ilo, C & Ajala, O 2024, 'Digitally enabled business\nmodels for a circular plastic economy in Africa', Environmental Technology &\nInnovation, vol. 35, 103657.\nhttps://dx.doi.org/10.1016/j.eti.2024.103657\nDOI 10.1016/j.eti.2024.103657\nISSN 2352-1864\nPublisher: Elsevier\nThis is an open access article under the CC BY license\n(http://creativecommons.org/licenses/by/4.0/)", metadata={'source': '/Users/admin/Documents/pdf_file_blobs/-files-94614918-Ajala2024VoR.pdf', 'file_path': '/Users/admin/Documents/pdf_file_blobs/-files-94614918-Ajala2024VoR.pdf', 'page': 0, 'total_pages': 17, 'Author': 'Kolade et al', 'CreationDate': "D:20240529140251+01'00'", 'Creator': 'Adobe Acrobat Pro DC (64-bit) 21.1.20135', 'ModDate':

In [22]:
from langchain_community.embeddings import OllamaEmbeddings

vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 30/30 [00:05<00:00,  5.11it/s]


In [27]:
# Use Ollama model of choice

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

cached_llm = Ollama(model="llama3")

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    cached_llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [28]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | cached_llm
    | StrOutputParser()
)

In [29]:
chain.invoke(input(" "))

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 27.21it/s]


'This document appears to be a research paper or report on "Digitally enabled business models for a circular plastic economy in Africa". It discusses the challenges and opportunities of transitioning to a circular economy, particularly in the context of plastic waste management in Africa. The document also presents various digital innovations that can help solve these challenges and create value in the circular plastic economy.'